In [1]:
import requests
import csv
import os
import pandas as pd
import json
import re
import time
from datetime import datetime, timedelta
import math
import traceback

In [3]:
input_path = "C:\\Users\\khan32\\Documents\\pr_fg_exchange\\"
output_path = input_path

run this query on personendatenbank

```sql
SELECT persons.factgrid, persons.id, gsn.nummer, items.status, items.deleted, persons.deleted, gsn.deleted
FROM items INNER JOIN persons ON persons.item_id = items.id
INNER JOIN gsn ON gsn.item_id = items.id;
```
```sql
SELECT persons.factgrid, persons.id, gsn.nummer, gsn.deleted 
FROM items 
INNER JOIN persons ON persons.item_id = items.id 
INNER JOIN gsn ON gsn.item_id = items.id;
```

In [4]:
filename = 'persons_all_2024-07-03.csv'

In [27]:
pr_df = pd.read_csv(os.path.join(input_path, filename), names=["fg_id", "id", "pd_id", 'is_deleted'])
pr_df

,fg_id,id,pd_id,is_deleted
0,NaN,4537,003-00001-001,0
1,NaN,4538,003-00002-001,0
2,NaN,4539,003-00003-001,0
3,NaN,4540,003-00004-001,0
4,NaN,4541,003-00005-001,0
...,...,...,...,...
382646,Q653267,336406,007-03087-001,0
382647,NaN,336408,027-04097-001,0
382648,NaN,336408,033-02524-001,0
382649,Q654054,336410,060-02172-001,0


In [33]:
url = 'https://database.factgrid.de/sparql'
query = (
"""SELECT ?item ?prid WHERE {
  ?item wdt:P472 ?prid.
}""")
# SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_df)

5405

In [34]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [35]:
drop_type_columns(factgrid_df)
extract_qid(factgrid_df, 'item.value')
factgrid_df.columns = ['FactGrid_ID', 'pd_id']
factgrid_df

,FactGrid_ID,pd_id
0,Q652063,059-00013-001
1,Q652072,005-00027-001
2,Q652112,006-00006-001
3,Q652104,014-00007-001
4,Q652076,035-00015-001
...,...,...
5400,Q948988,083-01861-001
5401,Q948989,083-01921-001
5402,Q948991,083-02366-001
5403,Q948993,005-00125-001


In [52]:
joined_df = pr_df.merge(factgrid_df, left_on='fg_id', right_on='FactGrid_ID', suffixes=('_pr', '_fg'))
joined_df

,fg_id,id,pd_id_pr,is_deleted,FactGrid_ID,pd_id_fg
0,Q653979,4719,003-00183-001,0,Q653979,003-00183-001
1,Q652353,4805,003-00269-001,0,Q652353,003-00269-001
2,Q652429,4904,003-00368-001,0,Q652429,003-00368-001
3,Q654339,4918,003-00382-001,0,Q654339,003-00382-001
4,Q656071,5147,003-00611-001,0,Q656071,003-00611-001
...,...,...,...,...,...,...
22806,Q653267,336406,015-05023-001,0,Q653267,030-04280-001
22807,Q653267,336406,700-00428-001,0,Q653267,030-04280-001
22808,Q653267,336406,007-03087-001,0,Q653267,030-04280-001
22809,Q654054,336410,060-02172-001,0,Q654054,060-02172-001


In [58]:
unequal_df = joined_df[joined_df['is_deleted'] == 0]
unequal_df

,fg_id,id,pd_id_pr,is_deleted,FactGrid_ID,pd_id_fg
0,Q653979,4719,003-00183-001,0,Q653979,003-00183-001
1,Q652353,4805,003-00269-001,0,Q652353,003-00269-001
2,Q652429,4904,003-00368-001,0,Q652429,003-00368-001
3,Q654339,4918,003-00382-001,0,Q654339,003-00382-001
4,Q656071,5147,003-00611-001,0,Q656071,003-00611-001
...,...,...,...,...,...,...
22806,Q653267,336406,015-05023-001,0,Q653267,030-04280-001
22807,Q653267,336406,700-00428-001,0,Q653267,030-04280-001
22808,Q653267,336406,007-03087-001,0,Q653267,030-04280-001
22809,Q654054,336410,060-02172-001,0,Q654054,060-02172-001


In [54]:
# what is wrong with Q653473 	300460 	044-03599-001 	1 	Q653473 	044-03599-001 ?????????????????????????????
# query on sql is also strange with 1 1 1 for deleted

In [59]:
unequal_df[unequal_df['pd_id_pr'] != unequal_df['pd_id_fg']]

,fg_id,id,pd_id_pr,is_deleted,FactGrid_ID,pd_id_fg
254,Q721448,308993,025-00973-001,0,Q721448,026-00947-001
257,Q723884,308547,026-01674-001,0,Q723884,061-01578-001
258,Q723884,308547,051-01545-001,0,Q723884,061-01578-001
259,Q723884,308547,048-03972-001,0,Q723884,061-01578-001
260,Q723884,308547,058-01279-001,0,Q723884,061-01578-001
...,...,...,...,...,...,...
22804,Q654401,336382,007-02354-001,0,Q654401,007-02350-001
22806,Q653267,336406,015-05023-001,0,Q653267,030-04280-001
22807,Q653267,336406,700-00428-001,0,Q653267,030-04280-001
22808,Q653267,336406,007-03087-001,0,Q653267,030-04280-001


In [60]:
# figure out how to deal with Q721448 	308993 	025-00973-001 	0 	Q721448 	026-00947-001